# Testing MultiSigBTC

##### Some imports we need for the testing

In [1]:
import json

##### And some variable

In [75]:
path_john = "output\\john.json"
path_bob = "output\\bob.json"
path_maria = "output\\maria.json"
path_alice = "output\\alice.json"
path_multisig_2_3 = "output\\multisig-2-3.json"
path_multisig_2_3_transaction = "output\\multisig-2-3-transaction.json"
path_multisig_2_2 = "output\\multisig-2-2.json"
path_multisig_2_2_transaction = "output\\multisig-2-2-transaction.json"
path_multisig_2_3_transact_2 = "output\\multisig-2-3-transaction_2.json"

### Update bitcoin.conf

<p>Before continue, you need to run a bitcoin node in regtest and enable server in configuration.<br>
To do so, we need to add the lines bellow in the bitcoin configuration (we use string 'admin' as the rpc use and its password)</p>

>regtest=1<br>
>server=1<br>
>rpcuser=admin<br>
>rpcpassword=admin<br>

**/!\\** *After we update the configuration file, we must execute the command **bitcoind** in a terminal/command line before we continue with the test*

## Prepare our regtest blockchain

Let's create a wallet and mine some blocks to get some bitcoin flowing in our blockchain

In [3]:
!bitcoin-cli createwallet wallet

{
  "name": "wallet",
  "warning": ""
}


In [4]:
miner = !bitcoin-cli getnewaddress address legacy
miner = miner[0]
miner

'mju2L9JRZ2TLGo8Xrx96TzavzGrMahiazb'

In [5]:
!bitcoin-cli generatetoaddress 101 {miner}

[
  "2956f93273e67d7945646b2b67935a7416a1f3ce89212f086c7b095f6d786faa",
  "2a34c688240a3049f7107f0eee54463ec0d126daf30d4d1767b042a40e6304c8",
  "1be952a07200b0541fc7543f5f550b887f59a25b56078ea6f8ec2830a4ca968b",
  "12fac1552f5868da40cdf8071580b7a5845e2e9b97e3f3d56f64cfb191cd5f97",
  "2212f98a86c289854fb2a7cdb502f9d5730e3bc33063c77e41d5fda633f9a91d",
  "38803d9ddc0206f3a22f953e2fbf1ac79de553fb9c0870e1649473dcd107c148",
  "3bdd4e7e8761ef4051a765f658cdf6de9b73bb1af83fad516fa3f061af8450ce",
  "5dfbc73d04ff415b777e736df9329f4a18e99a269b51d9a9635c088fc5c0f111",
  "482ca06f893260625ac291224a6e180ff64488dc398d881cca13772cae2a088d",
  "1168ba07ff6d6ad4d34ee01008dd9a03799cf6b30f4e1698edd727604c2746c4",
  "2c30c0147382b6a9e289e814d2853def15ff55689ea18ecebe44ad8ed0b484e1",
  "14bbc5141b3be6323c68f161cbcf9746143a4eeb44642e27e604c9712bf0ffa8",
  "600afbd17bab5876321bae1954ddfefb89aaac7c08885b6d3fed1db981f615e3",
  "2826bef6b37c6fd5b90f6ab5b65b6859e31579b1bd6ec1ee7fb0465a087833a3",
  "79e3adda2184f29

In [6]:
!bitcoin-cli listaddressgroupings

[
  [
    [
      "mju2L9JRZ2TLGo8Xrx96TzavzGrMahiazb",
      50.00000000,
      "address"
    ]
  ]
]


Ok so we created a wallet and an address and mined 101 blocks in order to get some funds in our address to work with (miner reward is locked for 100 blocks)

## Create some private-public key pairs to work with

##### John's Keys

In [7]:
result = !python.exe MultiSigBTC.py -o {path_john} create_new_address "John" "pass" -s "john"
#Store some values we will use later :
result = ''.join([str(elem) for elem in result]).replace("'",'"')
result = json.loads(result)
john = result['address']
result

{'address': 'mwiL5k3NeRCPAe2c5LDNZKESMNTsxT3qS7',
 'keys': {'name': 'John',
  'private': 'yUQyOn/lVvxBUZ7+jSGIIANKbiHcN/ZFskSKIssYFGnzGij1ltgjCEt1Dg8iDZ9NNqi4fq7GRfYAPe1MkBETzanjyRfop8ZdFMXnj1anQoe5z3UNIarYnT0km/bgpTogUeJ/xS4vlq4=',
  'public': '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c'},
 'type': 'P2PKH'}

##### Bob's Keys

In [8]:
!python.exe MultiSigBTC.py -o {path_bob} create_new_address "Bob" "pass" -s "bob"

{'address': 'msXtCoKiBft566TaYP9xjMAbRe1dPdSrSj',
 'keys': {'name': 'Bob',
          'private': 'PZRZJ9dWGYyqMGv7kMIn39GOVpm1dDQa8mfnj1AqrhnJTSzSm5nWmzLr0ralDSkHRt1D8sd5DTjWF5u0qZiYjrC4EhDKVT0hP6c5ml2z32I/cu7Kx/KeJj5DJVq6uXnOm2B084I6nls=',
          'public': '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c'},
 'type': 'P2PKH'}


##### Maria's Keys

In [9]:
!python.exe MultiSigBTC.py -o {path_maria} create_new_address "Maria" "pass" -s "maria"

{'address': 'mpRRNUdNSXYnK8enrzZzqzURGp8SqQfpPF',
 'keys': {'name': 'Maria',
          'private': 'ufNMY5Pb0m6ey+vX4+YG8nnyefAaYVdMagCmkMfxDbNxHHc1LnfwGMnFlxfiQb/QYau4Uxs/56fr0fVbLJurkCiMRbpgtRlh4PCd+nvGRpvK4Wzys224rDcEi1oUeyGl2qAhKRGz6/E=',
          'public': '02b6ec4ee5a73f82c69ac37cb27e8212cd0fef3e48768c7ca71ef3b82b344510bf'},
 'type': 'P2PKH'}


##### Alice's Keys

In [10]:
!python.exe MultiSigBTC.py -o {path_alice} create_new_address "Alice" "pass" -s "alice"

{'address': 'mkuWPd28Qn2R2mBUtDxbikYCxALKJujDoq',
 'keys': {'name': 'Alice',
          'private': '/uI8j7Q5reugnGGg327RVGc0rTNAt09Y8sGqBMYw+yi3vPIfL5IJ1nPZG43HMhVK9qZ0qXo6IUZjrIcx7lcOTuY8rfnZs2b5YdS/mnPwiCh7Bmyt12/f+4GXCjFsrsOKFYWkAcHTkhM=',
          'public': '039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc'},
 'type': 'P2PKH'}


## Create a 2 out of 3 Multi Signature Address

In [11]:
result = !python MultiSigBTC.py -o {path_multisig_2_3} create_multisig_address 2 {path_john} {path_bob} {path_alice}
#Store some values we will use later :
result = ''.join([str(elem) for elem in result]).replace("'",'"').replace("None",'""')
result = json.loads(result)
multisig_2_3 = result['address']
result

{'address': '2N8ZjuyrxVVHNDfLUKAQshJ4g2bvNkCMaeE',
 'keys': [{'name': 'John',
   'private': '',
   'public': '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c'},
  {'name': 'Bob',
   'private': '',
   'public': '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c'},
  {'name': 'Alice',
   'private': '',
   'public': '039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc'}],
 'script': [2,
  '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c',
  '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c',
  '039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc',
  3,
  'OP_CHECKMULTISIG'],
 'signatures_required': 2,
 'type': 'multisig_address'}

Now we have our 2 out of 3 multi signature address.

### Send some funds to the newly created multi signature address

In [12]:
!bitcoin-cli sendtoaddress {multisig_2_3} 1

143c86821894b07ef09f8bb1e78f6f32591889ecf6dce35294245fed05738cb8


In [13]:
!bitcoin-cli sendtoaddress {multisig_2_3} 0.3

c634a02a841e1f4ac40b2f425d9d1a68048e59ff6362c89981305230356e56b0


In [14]:
!bitcoin-cli sendtoaddress {multisig_2_3} 1.1

6c310433485947f1278ccdbcdc19db7b60f094a3c87273140f25c9862c9e7f71


In [15]:
!bitcoin-cli sendtoaddress {multisig_2_3} 5

650c996274cf200a0845b97cd2b665031010198cbd58ba4842e5ea5251f6c238


In [16]:
!bitcoin-cli generatetoaddress 1 {miner}

[
  "050119016316acb5e0499a0fe83fc602342b887abc5f4a6ea3df0fcfc99b8761"
]


In [17]:
!bitcoin-cli sendtoaddress {multisig_2_3} 52

76d239530a434d69c037de5ad8dbab8564254105b98408f63bb4c82f777886d6


In [18]:
!bitcoin-cli generatetoaddress 1 {miner}

[
  "0186b9e0361a13aef92df40537a93efb86704fc6e6c0cbb93985457789786f23"
]


To make it a little more challenging for our script (MultiSigBTC.py), we send multiple transactions in order for our multi signature address to own multiple UTXOs

### Let's check the balance of our address now

In [19]:
!python MultiSigBTC.py -u "admin" -p "admin" get_address_balance {path_multisig_2_3}

{'UTXOs': [{'amount': 52.0,
            'txid': '76d239530a434d69c037de5ad8dbab8564254105b98408f63bb4c82f777886d6',
            'vout': 1},
           {'amount': 5.0,
            'txid': '650c996274cf200a0845b97cd2b665031010198cbd58ba4842e5ea5251f6c238',
            'vout': 1},
           {'amount': 1.1,
            'txid': '6c310433485947f1278ccdbcdc19db7b60f094a3c87273140f25c9862c9e7f71',
            'vout': 1},
           {'amount': 1.0,
            'txid': '143c86821894b07ef09f8bb1e78f6f32591889ecf6dce35294245fed05738cb8',
            'vout': 0},
           {'amount': 0.3,
            'txid': 'c634a02a841e1f4ac40b2f425d9d1a68048e59ff6362c89981305230356e56b0',
            'vout': 1}],
 'balance': 59.4,
 'type': 'balance'}


### Now that we have some funds, let's create a transaction

We will try to send 58 BTC to John's address in order to control alone those 58 BTC, with his keys

In [20]:
!python MultiSigBTC.py -u "admin" -p "admin" -o {path_multisig_2_3_transaction} create_multisig_transaction {john} 58 0.01 {path_multisig_2_3}

{'keys': [{'name': 'John',
           'private': None,
           'public': '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c'},
          {'name': 'Bob',
           'private': None,
           'public': '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c'},
          {'name': 'Alice',
           'private': None,
           'public': '039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc'}],
 'script': [2,
            '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c',
            '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c',
            '039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc',
            3,
            'OP_CHECKMULTISIG'],
 'signatures_required': 2,
 'type': 'multisig_transaction',
 'unsigned_transaction': '0200000003d68678772fc8b43bf60884b90541256485abdbd85ade37c0694d430a5339d2760100000000ffffffff38c2f65152eae54248ba58bd8c1910100365b6d27cb945080a20cf7462990c650100000

### Let's try to sign the transaction two of the owners

In [21]:
!python MultiSigBTC.py -o {path_multisig_2_3_transaction} sign_multisig_transaction {path_multisig_2_3_transaction} {path_bob} "pass"

{'keys': [{'name': 'John',
           'private': None,
           'public': '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c'},
          {'name': 'Bob',
           'private': None,
           'public': '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c'},
          {'name': 'Alice',
           'private': None,
           'public': '039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc'}],
 'script': [2,
            '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c',
            '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c',
            '039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc',
            3,
            'OP_CHECKMULTISIG'],
 'signatures': {'650c996274cf200a0845b97cd2b665031010198cbd58ba4842e5ea5251f6c238': [{'name': 'Bob',
                                                                                      'signature': '304402206cecee069846f099f95645a26421193932347780

##### But what will happen if we try to send the transaction partially signed? Let's try it, why not...

In [22]:
!python MultiSigBTC.py get_signed_multisig_transaction {path_multisig_2_3_transaction}

{'exception': {'message': 'The transaction contains only 1 out of 2 signatures '
                          'required',
               'type': 'Exception'},
 'trace': [{'line': 192, 'name': '<module>'},
           {'line': 268, 'name': 'createdSignedTransaction'}],
 'type': 'exception'}


Ah yes I forgot about that, I have implemented a exemption when someone tries to get the signed transaction without having enough signatures... Ok I guess we will not send a partially signed transaction to the network :D

##### Ok let's sign the transaction also with Johns keys then

In [23]:
!python MultiSigBTC.py -o {path_multisig_2_3_transaction} sign_multisig_transaction {path_multisig_2_3_transaction} {path_john} "pass"

{'keys': [{'name': 'John',
           'private': None,
           'public': '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c'},
          {'name': 'Bob',
           'private': None,
           'public': '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c'},
          {'name': 'Alice',
           'private': None,
           'public': '039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc'}],
 'script': [2,
            '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c',
            '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c',
            '039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc',
            3,
            'OP_CHECKMULTISIG'],
 'signatures': {'650c996274cf200a0845b97cd2b665031010198cbd58ba4842e5ea5251f6c238': [{'name': 'John',
                                                                                      'signature': '3045022100e1a79736e05223af5b9d5aa53ef091bf30a82

We have now enough keys to created a signed transaction. Please note that it doesn’t really make sense to sign the transaction also with Alice’s key.<br>
MultiSigBTC is just keeping the signatures in the results and it will use just 2 out of 3 no matter if we provide 3 of them. Which 2 you ask?<br>
The first two. Of course this is not the best solution because one of the first two might be invalid in which case it will create an invalid transaction no matter if the third one is fine.<br>
This can be resolved by checking the validity of the signatures before use but let’s just say this part is work in progress :D

### Let's produce the signed transaction and submit it to the network

In [24]:
!python MultiSigBTC.py -o {path_multisig_2_3_transaction} get_signed_multisig_transaction {path_multisig_2_3_transaction}

0200000003d68678772fc8b43bf60884b90541256485abdbd85ade37c0694d430a5339d27601000000fdfe0000483045022100d0f121bdf635c83694dfbeca2d3f9f2abf372acf6494befae562ee275a6a3a1c022001e53fe3fae2928a2bcf477644ca39e17c5b2a1b69c3c6d841eb8b74b7c3ff4d014830450221008da1faa1573feffc82bee0b50a092b01f65f7db5b388d9df526d5fa2813566a6022006a5b734cf27eceeaf61dee0d33ccac687e5aa2432b84a93819ab31edb9d9885014c695221030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c21026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c21039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc53aeffffffff38c2f65152eae54248ba58bd8c1910100365b6d27cb945080a20cf7462990c6501000000fdfd0000483045022100e1a79736e05223af5b9d5aa53ef091bf30a829621a73b3c3c68fe18e80281e2702200fffe1b8596800511b1b2f84139f0c148965cd6d2cf5c9179d3fd7e03de2d93b0147304402206cecee069846f099f95645a26421193932347780cd9768e7409115917e2fe217022046a3d781e10355519a89e750f16cb1df01ce10a5be3e4c943db73c6054bcebdc014c695221030a10de3e8ea71824

In [25]:
!python MultiSigBTC.py -u "admin" -p "admin" send_transaction {path_multisig_2_3_transaction}

{'txid': ['18233a96afd0a34c75446fb9e54cc2dab0ed4c196457ae105e5b09e6fdd93398'],
 'type': 'txid'}


**Success!** The transaction has been submitted! We will generate a block now so it can be included in the blockchain and check the new balances

In [26]:
result = !bitcoin-cli generatetoaddress 1 {miner}
result = ''.join([str(elem) for elem in result]).replace("'",'"').replace("None",'""')
result = json.loads(result)
block = result[0]
block

'6c7c9440212a81504e7b661d8143f3e00ef53f449de731b9b8e7aadbfb2899f2'

In [27]:
!bitcoin-cli getblock {block}

{
  "hash": "6c7c9440212a81504e7b661d8143f3e00ef53f449de731b9b8e7aadbfb2899f2",
  "confirmations": 1,
  "height": 104,
  "version": 536870912,
  "versionHex": "20000000",
  "merkleroot": "3c4a488c53e14e8e1e363d4ade557a8526b5b0f996937c9c982fb35d1c1d1f30",
  "time": 1650129469,
  "mediantime": 1650129444,
  "nonce": 0,
  "bits": "207fffff",
  "difficulty": 4.656542373906925e-10,
  "chainwork": "00000000000000000000000000000000000000000000000000000000000000d2",
  "nTx": 2,
  "previousblockhash": "0186b9e0361a13aef92df40537a93efb86704fc6e6c0cbb93985457789786f23",
  "strippedsize": 1183,
  "size": 1219,
  "weight": 4768,
  "tx": [
    "92f2634c56ffdb3ef4542b40b528a4c5f50be897c516dfeb506d2938a094a67b",
    "18233a96afd0a34c75446fb9e54cc2dab0ed4c196457ae105e5b09e6fdd93398"
  ]
}


##### What is the balance of our multisig address now?

In [28]:
!python MultiSigBTC.py -u "admin" -p "admin" get_address_balance {path_multisig_2_3}

{'UTXOs': [{'amount': 1.0,
            'txid': '143c86821894b07ef09f8bb1e78f6f32591889ecf6dce35294245fed05738cb8',
            'vout': 0},
           {'amount': 0.3,
            'txid': 'c634a02a841e1f4ac40b2f425d9d1a68048e59ff6362c89981305230356e56b0',
            'vout': 1},
           {'amount': 0.09,
            'txid': '18233a96afd0a34c75446fb9e54cc2dab0ed4c196457ae105e5b09e6fdd93398',
            'vout': 1}],
 'balance': 1.39,
 'type': 'balance'}


It seems correct! The funds have been subtracted from the balance

##### And what about the balance on John's address we send the funds to?

In [29]:
!python MultiSigBTC.py -u "admin" -p "admin" get_address_balance {john}

{'UTXOs': [{'amount': 58.0,
            'txid': '18233a96afd0a34c75446fb9e54cc2dab0ed4c196457ae105e5b09e6fdd93398',
            'vout': 0}],
 'balance': 58.0,
 'type': 'balance'}


Everything looks into place! **The test was a success!**

## Create a 2 out of 2 multi signature address

Now that we have some thing already up and working from our previous test, let's try to create another address that needs 3 out of the 4 owners to sign the transactions

### Create the multi signature address

In [30]:
result = !python MultiSigBTC.py -o {path_multisig_2_2} create_multisig_address 2 {path_bob} {path_maria}
#Store some values we will use later :
result = ''.join([str(elem) for elem in result]).replace("'",'"').replace("None",'""')
result = json.loads(result)
multisig_2_2 = result['address']
result

{'address': '2MwRcPStyKRCE2WEsDcuUB84nrzqkkodC2a',
 'keys': [{'name': 'Maria',
   'private': '',
   'public': '02b6ec4ee5a73f82c69ac37cb27e8212cd0fef3e48768c7ca71ef3b82b344510bf'},
  {'name': 'Bob',
   'private': '',
   'public': '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c'}],
 'script': [2,
  '02b6ec4ee5a73f82c69ac37cb27e8212cd0fef3e48768c7ca71ef3b82b344510bf',
  '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c',
  2,
  'OP_CHECKMULTISIG'],
 'signatures_required': 2,
 'type': 'multisig_address'}

### Send some funds to the newly created address

In [31]:
!bitcoin-cli sendtoaddress {multisig_2_2} 14

480ad4939a8b224245262b5b9b9e848f92e7a2d45fb511595b20dfbfb0d971ef


In [32]:
!bitcoin-cli generatetoaddress 1 {miner}

[
  "5f96f8953cd8658974fbd2248af9a202623c6c7541c28b822b942025d9096aed"
]


In [33]:
!python MultiSigBTC.py -u "admin" -p "admin" get_address_balance {path_multisig_2_2}

{'UTXOs': [{'amount': 14.0,
            'txid': '480ad4939a8b224245262b5b9b9e848f92e7a2d45fb511595b20dfbfb0d971ef',
            'vout': 0}],
 'balance': 14.0,
 'type': 'balance'}


### We will now send some funds to the other multi signature address we created earlier

##### Firstly let's try sending more funds than we have to see what happens

In [34]:
!python MultiSigBTC.py -u "admin" -p "admin" -o {path_multisig_2_2_transaction} create_multisig_transaction {path_multisig_2_3} 15 0.01 {path_multisig_2_2}

{'exception': {'message': 'Not enough funds. Total amount is 14.0',
               'type': 'Exception'},
 'trace': [{'line': 175, 'name': '<module>'},
           {'line': 132, 'name': 'createMultisigTransaction'},
           {'line': 100, 'name': 'createTransaction'}],
 'type': 'exception'}


Great! The script works as expected, it gets the balance and return an error when we try to send more than we can.

##### Now we create a valid transaction and we will try to spend the whole UTXO we have (meaning no UTXO returning funds to our address)

In [35]:
!python MultiSigBTC.py -u "admin" -p "admin" -o {path_multisig_2_2_transaction} create_multisig_transaction {path_multisig_2_3} 13.99 0.01 {path_multisig_2_2}

{'keys': [{'name': 'Maria',
           'private': None,
           'public': '02b6ec4ee5a73f82c69ac37cb27e8212cd0fef3e48768c7ca71ef3b82b344510bf'},
          {'name': 'Bob',
           'private': None,
           'public': '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c'}],
 'script': [2,
            '02b6ec4ee5a73f82c69ac37cb27e8212cd0fef3e48768c7ca71ef3b82b344510bf',
            '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c',
            2,
            'OP_CHECKMULTISIG'],
 'signatures_required': 2,
 'type': 'multisig_transaction',
 'unsigned_transaction': '0200000001ef71d9b0bfdf205b5911b55fd4a2e7928f849e9b5b2b264542228b9a93d40a480000000000ffffffff01c00b63530000000017a914a80a5ad975825e6ea42dbd841a26c753851810d58700000000'}


So far so good! :D

### Now it's time to sign the transaction

In [36]:
!python MultiSigBTC.py -o {path_multisig_2_2_transaction} sign_multisig_transaction {path_multisig_2_2_transaction} {path_bob} "pass"

{'keys': [{'name': 'Maria',
           'private': None,
           'public': '02b6ec4ee5a73f82c69ac37cb27e8212cd0fef3e48768c7ca71ef3b82b344510bf'},
          {'name': 'Bob',
           'private': None,
           'public': '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c'}],
 'script': [2,
            '02b6ec4ee5a73f82c69ac37cb27e8212cd0fef3e48768c7ca71ef3b82b344510bf',
            '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c',
            2,
            'OP_CHECKMULTISIG'],
 'signatures': {'480ad4939a8b224245262b5b9b9e848f92e7a2d45fb511595b20dfbfb0d971ef': [{'name': 'Bob',
                                                                                      'signature': '3045022100ea90a4f8d5de721c40e150a3b7e7d800a6c26d589f36b9e39cf0b87b76e0671e02207e259bfe46a3fd095be08f994e64ada824ea705f8b53ec3273b4ebe11ab3011001'}]},
 'signatures_required': 2,
 'type': 'multisig_transaction',
 'unsigned_transaction': '0200000001ef71d9b0bfdf205b5911b55fd4a2e79

In [37]:
!python MultiSigBTC.py -o {path_multisig_2_2_transaction} sign_multisig_transaction {path_multisig_2_2_transaction} {path_maria} "passs"

{'exception': {'message': 'Failed to decrypt the private key',
               'type': 'Exception'},
 'trace': [{'line': 184, 'name': '<module>'},
           {'line': 203, 'name': 'signMultisigTransaction'},
           {'line': 251, 'name': 'getPrivateKey'}],
 'type': 'exception'}


Ops, wrong password... Did I do it on purpose? Maybe :D

In [38]:
!python MultiSigBTC.py -o {path_multisig_2_2_transaction} sign_multisig_transaction {path_multisig_2_2_transaction} {path_maria} "pass"

{'keys': [{'name': 'Maria',
           'private': None,
           'public': '02b6ec4ee5a73f82c69ac37cb27e8212cd0fef3e48768c7ca71ef3b82b344510bf'},
          {'name': 'Bob',
           'private': None,
           'public': '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c'}],
 'script': [2,
            '02b6ec4ee5a73f82c69ac37cb27e8212cd0fef3e48768c7ca71ef3b82b344510bf',
            '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c',
            2,
            'OP_CHECKMULTISIG'],
 'signatures': {'480ad4939a8b224245262b5b9b9e848f92e7a2d45fb511595b20dfbfb0d971ef': [{'name': 'Maria',
                                                                                      'signature': '30440220079365165c913b6241a5fa558cdfc9aaab84b1903f4aa569dc2ed83bbad1eaad02205e5e6247532e84e9923f28d3ff37bd58ed7889e52961729bffeec1c02d1dae0401'},
                                                                                     {'name': 'Bob',
                            

### Finally let's send the transaction and check the balances

In [39]:
tx = !python MultiSigBTC.py get_signed_multisig_transaction {path_multisig_2_2_transaction}
tx[0]

'0200000001ef71d9b0bfdf205b5911b55fd4a2e7928f849e9b5b2b264542228b9a93d40a4800000000da004730440220079365165c913b6241a5fa558cdfc9aaab84b1903f4aa569dc2ed83bbad1eaad02205e5e6247532e84e9923f28d3ff37bd58ed7889e52961729bffeec1c02d1dae0401483045022100ea90a4f8d5de721c40e150a3b7e7d800a6c26d589f36b9e39cf0b87b76e0671e02207e259bfe46a3fd095be08f994e64ada824ea705f8b53ec3273b4ebe11ab301100147522102b6ec4ee5a73f82c69ac37cb27e8212cd0fef3e48768c7ca71ef3b82b344510bf21026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c52aeffffffff01c00b63530000000017a914a80a5ad975825e6ea42dbd841a26c753851810d58700000000'

In [40]:
!python MultiSigBTC.py -u "admin" -p "admin" send_transaction {tx[0]}

{'txid': ['d439b6bf04da0f851eecc53e45708f17d3ffb383613305bd52d55f2f31fd1041'],
 'type': 'txid'}


Perfect the transaction has gone through!

### Let's put the transaction in a block and check the new balances

In [41]:
!bitcoin-cli generatetoaddress 1 {miner}

[
  "64eb2c994bcca1751dad34f6bbb0ec8f866e10bc811cbe40e589f86e08696e45"
]


In [42]:
!python MultiSigBTC.py -u "admin" -p "admin" get_address_balance {path_multisig_2_2}

{'balance': 0.0, 'type': 'balance'}


In [68]:
!python MultiSigBTC.py -u "admin" -p "admin" get_address_balance {path_multisig_2_3}

{'UTXOs': [{'amount': 13.99,
            'txid': 'd439b6bf04da0f851eecc53e45708f17d3ffb383613305bd52d55f2f31fd1041',
            'vout': 0},
           {'amount': 1.0,
            'txid': '143c86821894b07ef09f8bb1e78f6f32591889ecf6dce35294245fed05738cb8',
            'vout': 0},
           {'amount': 0.3,
            'txid': 'c634a02a841e1f4ac40b2f425d9d1a68048e59ff6362c89981305230356e56b0',
            'vout': 1},
           {'amount': 0.09,
            'txid': '18233a96afd0a34c75446fb9e54cc2dab0ed4c196457ae105e5b09e6fdd93398',
            'vout': 1}],
 'balance': 15.38,
 'type': 'balance'}


In [74]:
!python MultiSigBTC.py -u "admin" -p "admin" -o {path_multisig_2_3_transaction_2} create_multisig_transaction {multisig_2_2} 10 0.024 {path_multisig_2_3}

{'exception': {'message': "Invalid paramater '{path_multisig_2_3}'",
               'type': 'Exception'},
 'trace': [{'line': 175, 'name': '<module>'},
           {'line': 121, 'name': 'createMultisigTransaction'}],
 'type': 'exception'}


Great! Everything looks into place!

## Try to send a transaction with wrong signatures

Ok now let's try to send some funds from the 2 out of 3 multi signature address to the 2 out of 2 multi signature address

### Let's create the transaction

In [77]:
!python MultiSigBTC.py -u "admin" -p "admin" -o {path_multisig_2_3_transact_2} create_multisig_transaction {multisig_2_2} 10 0.024 {path_multisig_2_3}

{'keys': [{'name': 'John',
           'private': None,
           'public': '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c'},
          {'name': 'Bob',
           'private': None,
           'public': '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c'},
          {'name': 'Alice',
           'private': None,
           'public': '039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc'}],
 'script': [2,
            '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c',
            '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c',
            '039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc',
            3,
            'OP_CHECKMULTISIG'],
 'signatures_required': 2,
 'type': 'multisig_transaction',
 'unsigned_transaction': '02000000014110fd312f5fd552bd05336183b3ffd3178f70453ec5ec1e850fda04bfb639d40000000000ffffffff0200ca9a3b0000000017a9142dd75c5da3f50cfc6bb64cc815a5fab072d5cdce87c0a2a

### And now sign it

In [80]:
!python MultiSigBTC.py -o {path_multisig_2_3_transact_2} sign_multisig_transaction {path_multisig_2_3_transact_2} {path_john} "pass"

{'keys': [{'name': 'John',
           'private': None,
           'public': '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c'},
          {'name': 'Bob',
           'private': None,
           'public': '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c'},
          {'name': 'Alice',
           'private': None,
           'public': '039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc'}],
 'script': [2,
            '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c',
            '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c',
            '039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc',
            3,
            'OP_CHECKMULTISIG'],
 'signatures': {'d439b6bf04da0f851eecc53e45708f17d3ffb383613305bd52d55f2f31fd1041': [{'name': 'John',
                                                                                      'signature': '304402200a645819438883caac15ef8eb37bcee7580a64d

And now we will try to sign with Maria's keys who is not part of the multi signature address

In [82]:
!python MultiSigBTC.py -o {path_multisig_2_3_transact_2} sign_multisig_transaction {path_multisig_2_3_transact_2} {path_maria} "pass"

{'keys': [{'name': 'John',
           'private': None,
           'public': '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c'},
          {'name': 'Bob',
           'private': None,
           'public': '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c'},
          {'name': 'Alice',
           'private': None,
           'public': '039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc'}],
 'script': [2,
            '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c',
            '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c',
            '039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc',
            3,
            'OP_CHECKMULTISIG'],
 'signatures': {'d439b6bf04da0f851eecc53e45708f17d3ffb383613305bd52d55f2f31fd1041': [{'name': 'Maria',
                                                                                      'signature': '3045022100f9ae4e5e3f086297aa63a27ff348b47b6f25

### We are now ready to send it to the network and see what happens

In [85]:
!python MultiSigBTC.py -u "admin" -p "admin" -o {path_multisig_2_3_transact_2} get_signed_multisig_transaction {path_multisig_2_3_transact_2}

02000000014110fd312f5fd552bd05336183b3ffd3178f70453ec5ec1e850fda04bfb639d400000000fdfd0000483045022100f9ae4e5e3f086297aa63a27ff348b47b6f252d9a33e89d12f7dd6d8ea8ddc27202207f9ed30394b126707872b8956f630e1bdb712c453bbd58be4541e269a458c1010147304402200a645819438883caac15ef8eb37bcee7580a64d5a95dfa8dd548a1f1042d08340220636fc49855ef2510d8e4c77581b27704363b5cf10192a929f3b5f1c845501bb6014c695221030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c21026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c21039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc53aeffffffff0200ca9a3b0000000017a9142dd75c5da3f50cfc6bb64cc815a5fab072d5cdce87c0a2a3170000000017a914a80a5ad975825e6ea42dbd841a26c753851810d58700000000


In [86]:
!python MultiSigBTC.py -u "admin" -p "admin" send_transaction {path_multisig_2_3_transact_2}

{'exception': {'message': '-26: mandatory-script-verify-flag-failed (Signature '
                          'must be zero for failed CHECK(MULTI)SIG operation)',
               'type': 'JSONRPCException'},
 'trace': [{'line': 206, 'name': '<module>'},
           {'line': 237, 'name': 'sendTransaction'},
           {'line': 203, 'name': 'sendTransaction'},
           {'line': 170, 'name': 'batch_'}],
 'type': 'exception'}


As expected the transaction failed to be submited to the blockchain because the script contains an invalid signature. So all good!

### Let's try to create again the transaction and sign it correctly

In [87]:
!python MultiSigBTC.py -u "admin" -p "admin" -o {path_multisig_2_3_transact_2} create_multisig_transaction {multisig_2_2} 10 0.024 {path_multisig_2_3}

{'keys': [{'name': 'John',
           'private': None,
           'public': '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c'},
          {'name': 'Bob',
           'private': None,
           'public': '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c'},
          {'name': 'Alice',
           'private': None,
           'public': '039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc'}],
 'script': [2,
            '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c',
            '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c',
            '039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc',
            3,
            'OP_CHECKMULTISIG'],
 'signatures_required': 2,
 'type': 'multisig_transaction',
 'unsigned_transaction': '02000000014110fd312f5fd552bd05336183b3ffd3178f70453ec5ec1e850fda04bfb639d40000000000ffffffff0200ca9a3b0000000017a9142dd75c5da3f50cfc6bb64cc815a5fab072d5cdce87c0a2a

In [88]:
!python MultiSigBTC.py -o {path_multisig_2_3_transact_2} sign_multisig_transaction {path_multisig_2_3_transact_2} {path_alice} "pass"

{'keys': [{'name': 'John',
           'private': None,
           'public': '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c'},
          {'name': 'Bob',
           'private': None,
           'public': '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c'},
          {'name': 'Alice',
           'private': None,
           'public': '039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc'}],
 'script': [2,
            '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c',
            '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c',
            '039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc',
            3,
            'OP_CHECKMULTISIG'],
 'signatures': {'d439b6bf04da0f851eecc53e45708f17d3ffb383613305bd52d55f2f31fd1041': [{'name': 'Alice',
                                                                                      'signature': '304402200ff5376ea2b90123e4c95619a86f4f9936b9e1

In [89]:
!python MultiSigBTC.py -o {path_multisig_2_3_transact_2} sign_multisig_transaction {path_multisig_2_3_transact_2} {path_bob} "pass"

{'keys': [{'name': 'John',
           'private': None,
           'public': '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c'},
          {'name': 'Bob',
           'private': None,
           'public': '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c'},
          {'name': 'Alice',
           'private': None,
           'public': '039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc'}],
 'script': [2,
            '030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c',
            '026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c',
            '039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc',
            3,
            'OP_CHECKMULTISIG'],
 'signatures': {'d439b6bf04da0f851eecc53e45708f17d3ffb383613305bd52d55f2f31fd1041': [{'name': 'Bob',
                                                                                      'signature': '3044022077be47a98b69487b3563a08ba523d1dabe5b464f

In [90]:
!python MultiSigBTC.py -u "admin" -p "admin" -o {path_multisig_2_3_transact_2} get_signed_multisig_transaction {path_multisig_2_3_transact_2}

02000000014110fd312f5fd552bd05336183b3ffd3178f70453ec5ec1e850fda04bfb639d400000000fc00473044022077be47a98b69487b3563a08ba523d1dabe5b464f1ab523383788946a4a018d000220699be5aae8f4ea16fae671f8881d386a8a3ce0c52497a479bcc5103646667e5b0147304402200ff5376ea2b90123e4c95619a86f4f9936b9e1fd78cb8769587c027f87d2456f02207bb5333573baac1acf818a860346b3061bed00f15a06eb0bde1aabb1835ffee7014c695221030a10de3e8ea718246cc79424d0f27fe7ec193e8949662a5cff75a23f49a6094c21026f7a4f6fcf1ae8aa25e1227cb9b67aa462edc47e956d7d3d7dbe899e09484e4c21039dabdd38f7eb2eff0dd7d8fadac724fa7f0330d03c267c0a6a7958143c49babc53aeffffffff0200ca9a3b0000000017a9142dd75c5da3f50cfc6bb64cc815a5fab072d5cdce87c0a2a3170000000017a914a80a5ad975825e6ea42dbd841a26c753851810d58700000000


In [91]:
!python MultiSigBTC.py -u "admin" -p "admin" send_transaction {path_multisig_2_3_transact_2}

{'txid': ['0bcccf63811557ffe9de76a2a4270126d4777fec23c7af624ca89046b87c6468'],
 'type': 'txid'}


In [92]:
!bitcoin-cli generatetoaddress 1 {miner}

[
  "025bc1085081b7be690ae788ce850501b73ed6b669e2f72b696e556c481e376b"
]


In [93]:
!python MultiSigBTC.py -u "admin" -p "admin" get_address_balance {multisig_2_2}

{'UTXOs': [{'amount': 10.0,
            'txid': '0bcccf63811557ffe9de76a2a4270126d4777fec23c7af624ca89046b87c6468',
            'vout': 0}],
 'balance': 10.0,
 'type': 'balance'}


In [94]:
!python MultiSigBTC.py -u "admin" -p "admin" get_address_balance {multisig_2_3}

{'UTXOs': [{'amount': 3.966,
            'txid': '0bcccf63811557ffe9de76a2a4270126d4777fec23c7af624ca89046b87c6468',
            'vout': 1},
           {'amount': 1.0,
            'txid': '143c86821894b07ef09f8bb1e78f6f32591889ecf6dce35294245fed05738cb8',
            'vout': 0},
           {'amount': 0.3,
            'txid': 'c634a02a841e1f4ac40b2f425d9d1a68048e59ff6362c89981305230356e56b0',
            'vout': 1},
           {'amount': 0.09,
            'txid': '18233a96afd0a34c75446fb9e54cc2dab0ed4c196457ae105e5b09e6fdd93398',
            'vout': 1}],
 'balance': 5.356,
 'type': 'balance'}


**Perfect!** The test concludes here! :)